In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data()
df["p-pet"] = df["precip"] - df["PET"]
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN,p-pet
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.112914,0.335343,0.354057,6052.592985,31.567899,17.204530,29.105844,24.562860,29.105844,-7.33
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.112904,0.335286,0.354033,6178.213349,29.570000,17.390000,29.760736,26.096551,29.760736,-6.52
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.112893,0.335229,0.354008,6313.892022,31.097908,17.235624,30.299442,25.986418,30.299442,-6.92
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.112882,0.335173,0.353984,6459.629005,30.868718,17.248525,28.978868,25.122209,28.978868,-6.35
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.112872,0.335116,0.353960,6615.424298,30.657792,17.259663,30.653667,27.830367,30.653667,-5.13


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 29 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_bnd1         1028 non-null float64
wp_bnd2         1028 non-null float64
wp_bnd3         1028 non-null float64
wp_bnd7         1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2", "wp_RNET", "wp_gpp", "p-pet"]
X, Y = exp.featurize(df, train_cols, ["wp_ch4_gf"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 15)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.890048221562
CV Scores:  0.887082467807, 0.915472745611, 0.919925507106, 0.880612504986, 0.885720740663, 0.870920363381, 0.876188213585, 0.861358847006, 0.884176511696, 0.919024313783
OOB score: 0.895099908122
Feature Importances:
('wp_gpp', 0.14266111752006524)
('wp_LST.night', 0.12684454476208018)
('wp_ndvi', 0.1226382582201764)
('air_temp', 0.091275492273427183)
('wp_lswi2', 0.089684076921122069)
('wp_LST.day', 0.089473320243890597)
('wp_evi', 0.082318235436756773)
('wp_RNET', 0.051180135449871794)
('p-pet', 0.044496370411775177)
('sw_in', 0.043388271284714112)
('PET', 0.038767744169195796)
('wind_speed', 0.029814300123966405)
('LW_IN.wp', 0.024345788976168424)
('VPD', 0.022256403564487531)
('precip', 0.00085594064230230999)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.867686882612
CV Scores:  0.900171240431, 0.912139441094, 0.893024916917, 0.86468068888, 0.85616948532, 0.854529176875, 0.835365168393, 0.803050364857, 0.858762719589, 0.89897562376
Feature Importances:
('wp_lswi2', 0.13169500361403158)
('wp_ndvi', 0.11916986020122147)
('wp_evi', 0.098940905253456501)
('wp_gpp', 0.092320912995040111)
('wp_LST.night', 0.091814775370416546)
('wp_LST.day', 0.069719173308859905)
('wind_speed', 0.062843420892641097)
('wp_RNET', 0.058646240065800805)
('LW_IN.wp', 0.057404744139206897)
('air_temp', 0.053974952259013477)
('VPD', 0.048218246993992631)
('sw_in', 0.044876850856725714)
('PET', 0.038020770528013358)
('p-pet', 0.030782855823740015)
('precip', 0.0015712876978398785)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.0762607093256
CV Scores:  -0.0738174480601, 0.0885820995091, 0.127697448142, 0.000383236353717, 0.111832793235, 0.0621069301324, 0.0867776909491, 0.176758003732, 0.0842207407245, 0.0980655985385


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [16]:
r.dnn_cross_val(X, Y, regr=r.dnn(nn_steps=10000, nn_lr=0.2, hidden_units=[50, 50]), k=10)

Running Neural Network Cross Validation...
Step #1, avg. loss: 972478.87500
Step #1001, epoch #100, avg. loss: 158981.70312
Step #2001, epoch #200, avg. loss: 97869.50781
Step #3001, epoch #300, avg. loss: 77708.04688
Step #4001, epoch #400, avg. loss: 68245.14844
Step #5001, epoch #500, avg. loss: 59790.46875
Step #6001, epoch #600, avg. loss: 51954.58594
Step #7001, epoch #700, avg. loss: 45593.22266
Step #8001, epoch #800, avg. loss: 41861.60547
Step #9001, epoch #900, avg. loss: 37770.65625
Step #1, avg. loss: 1277740.00000
Step #1001, epoch #100, avg. loss: 154434.68750
Step #2001, epoch #200, avg. loss: 92122.70312
Step #3001, epoch #300, avg. loss: 76219.69531
Step #4001, epoch #400, avg. loss: 64891.32812
Step #5001, epoch #500, avg. loss: 55986.75391
Step #6001, epoch #600, avg. loss: 49334.45703
Step #7001, epoch #700, avg. loss: 44876.83984
Step #8001, epoch #800, avg. loss: 41326.89453
Step #9001, epoch #900, avg. loss: 38516.38281
Step #1, avg. loss: 1340261.87500
Step #10

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.2,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x11a55d758>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=10000,
          tf_master='', tf_random_seed=42, verbose=1)